In [ ]:
import os
import cv2
import shutil
import numpy as np
from scipy.ndimage import filters

# Gaussian Blur function with dilated edges
def gaussian_blur(image, kernel_size=(5, 5), sigmaX=0):
    edges = cv2.Canny(image, 100, 200)
    dilated_edges = cv2.dilate(edges, None, iterations=2)
    dilated_edges = np.expand_dims(dilated_edges, axis=2)
    smoothed_image = cv2.GaussianBlur(image, kernel_size, sigmaX)
    smoothed_image[dilated_edges > 0] = image[dilated_edges > 0]
    return smoothed_image

# Bilateral Filter function with dilated edges
def bilateral_filter(image, d=9, sigma_color=75, sigma_space=75):
    edges = cv2.Canny(image, 100, 200)
    dilated_edges = cv2.dilate(edges, None, iterations=2)
    dilated_edges = np.expand_dims(dilated_edges, axis=2)
    smoothed_image = cv2.bilateralFilter(image, d, sigma_color, sigma_space)
    smoothed_image[dilated_edges > 0] = image[dilated_edges > 0]
    return smoothed_image

# Median Filter function with dilated edges
def median_filter(image, kernel_size=3):
    edges = cv2.Canny(image, 100, 200)
    dilated_edges = cv2.dilate(edges, None, iterations=2)
    dilated_edges = np.expand_dims(dilated_edges, axis=2)
    smoothed_image = cv2.medianBlur(image, kernel_size)
    smoothed_image[dilated_edges > 0] = image[dilated_edges > 0]
    return smoothed_image

# Anisotropic Diffusion function with dilated edges
def anisotropic_diffusion(image, n_iter=50, delta_t=0.1, kappa=100):
    edges = cv2.Canny(image, 100, 200)
    dilated_edges = cv2.dilate(edges, None, iterations=2)
    dilated_edges = np.expand_dims(dilated_edges, axis=2)

    for _ in range(n_iter):
        grad_x = filters.gaussian_gradient_magnitude(image.astype(np.float64), sigma=1.0)
        c = 1.0 / (1.0 + (grad_x / kappa) ** 2)
        laplacian = filters.laplace(c * grad_x).astype(np.float64)
        image = np.clip(image + delta_t * laplacian, 0, 255).astype(np.uint8)

    image[dilated_edges > 0] = image[dilated_edges > 0]
    return image

# Non-Local Means Filter function with dilated edges
def non_local_means(image, h=10, h_for_color=10, template_window_size=5, search_window_size=21):
    if len(image.shape) == 2:  # Grayscale image
        edges = cv2.Canny(image, 100, 200)
        dilated_edges = cv2.dilate(edges, None, iterations=2)
        dilated_edges = np.expand_dims(dilated_edges, axis=2)
        smoothed_image = cv2.fastNlMeansDenoising(image, None, h, template_window_size, search_window_size)
        smoothed_image[dilated_edges > 0] = image[dilated_edges > 0]
        return smoothed_image
    else:  # Color image
        edges = cv2.Canny(image, 100, 200)
        dilated_edges = cv2.dilate(edges, None, iterations=2)
        smoothed_image = cv2.fastNlMeansDenoisingColored(image, None, h_for_color, h, template_window_size, search_window_size)
        smoothed_image[dilated_edges > 0] = image[dilated_edges > 0]
        return smoothed_image

# Gradient-based method (Gradient Inverse Weighted Interpolation) function with dilated edges
def gradient_inverse_weighted_interpolation(image, alpha=0.2, threshold=30):
    edges = cv2.Canny(image, 100, 200)
    dilated_edges = cv2.dilate(edges, None, iterations=2)
    dilated_edges = np.expand_dims(dilated_edges, axis=2)
    
    # Implementation of gradient-based method goes here (as shown in the previous response)
    # You can apply the dilated edges mask to the smoothed_image here before returning

    return smoothed_image

# Function to apply an edge smoothing technique and save the result to a folder
def apply_edge_smoothing(original_folder, output_folder, smoothing_function, **params):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(original_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(original_folder, filename)
            original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Apply the edge smoothing technique
            smoothed_image = smoothing_function(original_image, **params)

            # Save the smoothed image to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, smoothed_image)

# Define the paths for the original segmented images folder and output folders for each technique
original_images_folder = "path/to/original_images_folder"
output_gaussian_folder = "path/to/output/gaussian"
output_bilateral_folder = "path/to/output/bilateral"
output_median_folder = "path/to/output/median"
output_anisotropic_folder = "path/to/output/anisotropic"
output_non_local_means_folder = "path/to/output/non_local_means"
output_gradient_based_folder = "path/to/output/gradient_based"

# Apply different edge smoothing techniques to the original images
apply_edge_smoothing(original_images_folder, output_gaussian_folder, gaussian_blur, kernel_size=(5, 5), sigmaX=0)
apply_edge_smoothing(original_images_folder, output_bilateral_folder, bilateral_filter, d=9, sigma_color=75, sigma_space=75)
apply_edge_smoothing(original_images_folder, output_median_folder, median_filter, kernel_size=3)
apply_edge_smoothing(original_images_folder, output_anisotropic_folder, anisotropic_diffusion, n_iter=50, delta_t=0.1, kappa=100)
apply_edge_smoothing(original_images_folder, output_non_local_means_folder, non_local_means, h=10, template_window_size=5, search_window_size=21)
apply_edge_smoothing(original_images_folder, output_gradient_based_folder, gradient_inverse_weighted_interpolation, alpha=0.2, threshold=30)
